In [11]:
from fastapi import FastAPI
from fastapi.staticfiles import StaticFiles
from fastapi.responses import FileResponse
import uvicorn
import pandas as pd
from sklearn.cluster import HDBSCAN
from alpha_shapes import Alpha_Shaper
import random
from polylabel import polylabel
import json

ALPHA = 0.5

hdbscan = HDBSCAN(min_cluster_size=10, cluster_selection_method='leaf')

def random_color(i = 4253766):
  if i == -1:
    return "#eee"
  random.seed(i)
  r = lambda: random.randint(0, 255)
  return '#%02X%02X%02X' % (r(), r(), r())

In [18]:
features_out = {}
  
# Test dataset
# data = pd.read_csv("public/data/mockdata.csv")

# Atlanta dataset
data = pd.read_csv("public/data/zillow_all_active_atlanta.csv")
data = data.rename(columns={'latitude': 'lat', 'longitude': 'lng'})

data = data.drop_duplicates(subset=['lng', 'lat'])
data = data.dropna(subset=['lng', 'lat'])
data['cluster'] = hdbscan.fit_predict(data[['lng', 'lat']])

# generate points
point_list = []
for i, row in data.iterrows():
  point = {
    "type": "Feature",
    "geometry": {
      "type": "Point",
      "coordinates": [row['lng'], row['lat']]
    },
    "properties": {
      "cluster": row['cluster'],
      "color": random_color(row['cluster'])
    }
  }
  point_list.append(point)
features_out['points'] = {
  "type": "FeatureCollection",
  "features": point_list
}

polygon_list = []
center_list = []
cluster_group_df = data.groupby('cluster')
for cluster, cluster_df in cluster_group_df:
  if cluster == -1:
    continue
  coords = cluster_df.apply(lambda x: (x['lng'], x['lat']), axis=1).tolist()
  if len(coords) < 3:
    continue
  shaper = Alpha_Shaper(coords)
  alpha_shape = shaper.get_shape(ALPHA)
  alpha_shape_coords = [list(coord) for coord in alpha_shape.exterior.coords]
  center_point = polylabel([alpha_shape_coords])
  center = {
    "type": "Feature",
    "geometry": {
      "type": "Point",
      "coordinates": [float(center_point[0]), float(center_point[1])]
    },
    "properties": {
      "cluster": cluster,
      "color": random_color(cluster)
    }
  }
  center_list.append(center)
  polygon = {
    "type": "Feature",
    "properties": {
      "cluster": cluster,
      "color": random_color(cluster)
    },
    "geometry": {
      "type": "Polygon",
      "coordinates": [
        alpha_shape_coords
      ]
    }
  }
  polygon_list.append(polygon)
features_out['centers'] = {
  "type": "FeatureCollection",
  "features": center_list
}
features_out['shapes'] = {
  "type": "FeatureCollection",
  "features": polygon_list
}
features_out

{'points': {'type': 'FeatureCollection',
  'features': [{'type': 'Feature',
    'geometry': {'type': 'Point', 'coordinates': [-84.25877, 33.776173]},
    'properties': {'cluster': -1, 'color': '#eee'}},
   {'type': 'Feature',
    'geometry': {'type': 'Point', 'coordinates': [-84.26334, 33.763985]},
    'properties': {'cluster': -1, 'color': '#eee'}},
   {'type': 'Feature',
    'geometry': {'type': 'Point', 'coordinates': [-84.25798, 33.775093]},
    'properties': {'cluster': -1, 'color': '#eee'}},
   {'type': 'Feature',
    'geometry': {'type': 'Point', 'coordinates': [-84.26875, 33.7665]},
    'properties': {'cluster': -1, 'color': '#eee'}},
   {'type': 'Feature',
    'geometry': {'type': 'Point', 'coordinates': [-84.25563, 33.78214]},
    'properties': {'cluster': -1, 'color': '#eee'}},
   {'type': 'Feature',
    'geometry': {'type': 'Point', 'coordinates': [-84.26283, 33.782543]},
    'properties': {'cluster': -1, 'color': '#eee'}},
   {'type': 'Feature',
    'geometry': {'type': 'P

In [19]:
json.dumps(features_out).encode('utf-8')

b'{"points": {"type": "FeatureCollection", "features": [{"type": "Feature", "geometry": {"type": "Point", "coordinates": [-84.25877, 33.776173]}, "properties": {"cluster": -1, "color": "#eee"}}, {"type": "Feature", "geometry": {"type": "Point", "coordinates": [-84.26334, 33.763985]}, "properties": {"cluster": -1, "color": "#eee"}}, {"type": "Feature", "geometry": {"type": "Point", "coordinates": [-84.25798, 33.775093]}, "properties": {"cluster": -1, "color": "#eee"}}, {"type": "Feature", "geometry": {"type": "Point", "coordinates": [-84.26875, 33.7665]}, "properties": {"cluster": -1, "color": "#eee"}}, {"type": "Feature", "geometry": {"type": "Point", "coordinates": [-84.25563, 33.78214]}, "properties": {"cluster": -1, "color": "#eee"}}, {"type": "Feature", "geometry": {"type": "Point", "coordinates": [-84.26283, 33.782543]}, "properties": {"cluster": -1, "color": "#eee"}}, {"type": "Feature", "geometry": {"type": "Point", "coordinates": [-84.262856, 33.78237]}, "properties": {"cluster